## Setup

In this notebook we extend the **channel–flow** example to **density-based topology optimisation (TO)** for an **exterior flow** (uniform freestream past a design region), targeting **drag minimisation**. For the forward Navier–Stokes–Brinkman implementation, refer to `solve_fluid.ipynb`.

We use the Brinkman body-force formulation; the drag objective is
$$
\min_{\gamma}\;\; J_{\mathrm{drag}}
\;=\;
\int_{\Omega} \alpha(\gamma)\,u_x\,d\Omega
\quad\text{s.t.}\quad
\mathbf{R}(\gamma,\mathbf{u},p)=\mathbf{0},\;\; g(\gamma)\le 0,
$$
where $\alpha(\gamma)$ is the design-dependent Brinkman penalty and $u_x$ is the streamwise velocity. The exterior-flow setup uses Dirichlet inflow ($\mathbf{u}=[U_{in}, 0]^T$), traction-free outflow, and far-field/symmetry lateral boundaries.

### Exterior-Flow Boundary Conditions

To illustrate **drag-minimizing** fluid TO, we consider a uniform freestream past a design region (exterior flow), as shown below:

<img src="../../figures/drag_bc.png" alt="Exterior-flow drag setup" style="width: 500px;"/>

- **Inflow (left):** Dirichlet, $\mathbf{u}=[U_{in}, 0]^T$
- **Outflow (right):** traction-free (zero-normal-stress)
- **Lateral boundaries:** far-field/symmetry (no spurious reflection)
- **Design domain $\Omega_d$:** embedded in the flow; Brinkman penalization is active only in $\Omega_d$

### Design parametrisation

We adopt the **element-wise density method**: the design vector $\gamma\in[0,1]$ is mapped to the Brinkman damping $\alpha(\gamma)$ via a **RAMP** scheme in the design domain $\Omega_d$ (optionally preceded by density filtering/projection).

### Optimiser

We use the **Method of Moving Asymptotes (MMA)**.
Gradients $\partial J/\partial\gamma$ and $\partial g/\partial\gamma$ are obtained automatically via JAX AD.

> **Convention note.** Following solid-mechanics TO, **solid** corresponds to pseudo-density $\gamma=1$ and **fluid/void** to $\gamma=0$ (the opposite convention is sometimes used in fluid TO).

In [ ]:
import functools

import numpy as np
import jax.numpy as jnp
import jax
from jax.typing import ArrayLike
import matplotlib.pyplot as plt

import toflux.src.geometry as _geom
import toflux.src.mesher as _mesher
import toflux.src.material as _mat
import toflux.src.utils as _utils
import toflux.src.bc as _bc
import toflux.src.fe_fluid as _fea
import toflux.src.solver as _solv
import toflux.src.mma as _mma
import toflux.src.viz as _viz

jax.config.update("jax_enable_x64", True)
plt.rcParams.update(_viz.default_plot_settings)

_Ext = _utils.Extent
_Field = _fea.FluidField
_cmap = _viz.fluid_cmap

# Define Geometry and Mesh

Below we build the **drag minimization** benchmark commonly used in fluid TO.

#### Geometry
A simple rectangular domain is described in a json and passed as the geometry. This geometry is passed to the mesher.

#### Discretisation
* The domain $(\Omega)$ is partitioned into **bilinear quadrilateral (Q1) elements**.
* Both primary fields—velocity $(\mathbf u=(u_x,u_y))$ **and** pressure (p)—are interpolated with the same Q1 shape functions (equal-order Q1/Q1 formulation).

#### Numerical integration
Element contributions (mass, convection, brinkman, viscous and stability terms) are evaluated with a $(2 \times 2)$ Gauss quadrature.

This mesh serves as the baseline for the finite-element Navier–Stokes solve and for any subsequent density-based topology-optimisation runs.

In [ ]:
geom = _geom.BrepGeometry(brep_file="toflux/brep/channel_drag.json")

mesh = _mesher.grid_mesh_brep(
  brep=geom,
  nelx_desired=210,
  nely_desired=70,
  dofs_per_node=3,
  gauss_order=2,
)

_viz.plot_brep(geom)

## Design Domain Region

Here we visualize the design region used for optimization. Rest of the region assumes fluid flow during optimization.

In [ ]:
lx, ly = mesh.bounding_box.lx, mesh.bounding_box.ly
dx, dy = mesh.elem_size[0], mesh.elem_size[1]
design_bbox = _mesher.BoundingBox(
  x=_Ext(min=0.2 * lx, max=0.7 * lx),
  y=_Ext(min=0.25 * ly, max=0.75 * ly),
)

design_bbox_elems = _mesher.compute_point_indices_in_box(mesh.elem_centers, design_bbox)
non_design_elems = jnp.logical_not(design_bbox_elems)
design_dom_vol = mesh.domain_volume * (design_bbox_elems.sum() / mesh.num_elems)

## Flow and Material Setup


For this fluid demonstration we keep the parameters dimensionless:

| Symbol | Meaning | Value |
|--------|---------|-------|
| $(\rho)$ | Mass density | **1** |
| $(\mu)$ | Dynamic viscosity | **1** |

We also prepare the *inverse permeability* field used in the Brinkman term.  
A RAMP interpolation converts the design variable $(\gamma\in[0,1])$ into the damping coefficient $(\alpha)$. In the subsequent topology optimization the brinkman penalty value damps fluid flow in solid.

In [ ]:
reynolds_number = 1.0
characteristic_velocity = 1.0
min_mat_frac = 0.15 * design_dom_vol/ mesh.domain_volume # mat fraction of the airfoil
characteristic_length = jnp.sqrt((1.0 - min_mat_frac) * design_dom_vol)

mass_density = 1.0
dynamic_viscosity = (
  characteristic_velocity * characteristic_length * mass_density / reynolds_number
)

mat_params = _mat.FluidMaterial(
  mass_density=1.0,
  dynamic_viscosity=dynamic_viscosity,
)

min_inv_permeability = 0.0
max_inv_permeability = mat_params.dynamic_viscosity * 1.0e5 / characteristic_length
inv_permeability_ext = _utils.Extent(min=min_inv_permeability, max=max_inv_permeability)

In [ ]:
init_design = jnp.zeros(mesh.num_elems)
init_design = init_design.at[design_bbox_elems].set(
  min_mat_frac * mesh.domain_volume / design_dom_vol
)
_viz.plot_grid_mesh(mesh, init_design)

## Boundary Conditions


| Region | Type | Imposed values |
|--------|------|----------------|
| **Inlet top and bottom(left vertical edge)** | Dirichlet | $(u = parabolic)$, $(v = 0)$ |
| **Outlet top and bottom (right vertical edge)** | Dirichlet v, p | \(p = 0\), \(v = 0\) |
| **Top wall** | No-slip (Dirichlet) | \(u = 0\), \(v = 0\) |
| **Bottom wall** | No-slip (Dirichlet) | \(u = 0\), \(v = 0\) |

**Characteristic velocity**

The inlet velocity is chosen to give a Reynolds number of 1:

$$
U_c \;=\; \frac{\mathrm{Re}\,\nu}{H},
\qquad
\text{Re} = 1,\;
\quad
\nu = \frac{\mu}{\rho},
\quad
H = \text{characteristic length}.
$$


In [ ]:
face_tol = mesh.elem_size[0] * 0.5

# open condition
open_faces = _bc.identify_faces(
  mesh, edges=[geom.edges[0], geom.edges[1], geom.edges[2], geom.edges[4]], tol=face_tol
)
n = len(open_faces)
u_vel = (_Field.U_VEL, characteristic_velocity * jnp.ones((n,)))
v_vel = (_Field.V_VEL, jnp.zeros((n,)))
open_face_val = [u_vel, v_vel]

# pressure condition
press_faces = _bc.identify_faces(mesh, edges=[geom.edges[3]], tol=face_tol)
n = len(press_faces)
pres = (_Field.PRESSURE, jnp.zeros(n))
u_vel = (_Field.U_VEL, characteristic_velocity * jnp.ones((n,)))
v_vel = (_Field.V_VEL, jnp.zeros((n,)))
pres_face_val = [u_vel, v_vel, pres]


open_bc = _bc.DirichletBC(elem_faces=open_faces, values=open_face_val, name="open")
press_bc = _bc.DirichletBC(elem_faces=press_faces, values=pres_face_val, name="pressure")


bc_list = [open_bc, press_bc]
bc = _bc.process_boundary_conditions(bc_list, mesh)

_viz.plot_bc(bc_list, mesh)

## Solver

The incompressible Navier–Stokes–Brinkman system is **intrinsically non-linear** because of the convective term .
Our solver therefore employs a *damped/modified Newton–Raphson* loop. For more details see 'solver.py'. While for structural the physics is linear, the solver still performs a Newton–Raphson step; for a small-strain model this step converges in **one iteration**. 

In [ ]:
solver_settings = {
  "linear": {
    "solver": _solv.LinearSolvers.PETSC,
    "rtol": 1.0e-5,
    "petsc_solver": {},
  },
  "nonlinear": {"max_iter": 10, "threshold": 1.0e-6},
}

# Initialize

We initialize the FEA solver with the mesh, material, boundary conditions and solver settings. We provide an inital guess of the solution with the Dirichlet condition enforced.

In [ ]:
flow_solver = _fea.FluidSolver(mesh, bc, mat_params, solver_settings=solver_settings)
init_press_vel = jnp.zeros((mesh.num_dofs,))
init_press_vel = init_press_vel.at[bc["fixed_dofs"]].set(bc["dirichlet_values"])

## Optimize

In [ ]:
obj_scale = 1.e3

@functools.partial(jax.jit, static_argnames=("flow_solver",))
def objective_function(
  mat_frac: jnp.ndarray,
  flow_solver: _fea.FluidSolver,
  ramp_penalty: float,
  init_press_vel=None,
):
  
  """Computes the objective function and its gradient.

  This function computes the objective as the total drag in the flow field,
    which is the sum of the drag in each element.

  Args:
    mat_frac: Material fraction array of shape (num_elems,). The array should contain 
      values between 0 and 1, representing the material fraction at each element. Where 
      0 indicates fluid and 1 indicates material. The material fraction can assume 
      intermediate values between 0 and 1 during optimization.
    flow_solver: Fluid solver instance.
    ramp_penalty: Penalty factor for the Brinkman interpolation using a convex ramp 
      function. The ramp penalty is  updated using a continuation scheme during
      optimization. In the beginning, the ramp penalty is set to a large value (which 
      makes the mat frac vs Brinkman penalty convex which can be determined by 
      brink_iter_factor) to allow fluid flow through the entire domain. As the 
      optimization progresses, the ramp penalty is reduced to allow the material 
      fraction to converge to material or fluid.
    init_press_vel: Initial pressure-velocity field of shape (num_dofs,). This is used 
      as the initial guess for the pressure-velocity field to Newton-Raphson
      iterations. It contains the Dirichlet boundary conditions applied to the
      pressure-velocity field.

  Returns:
    A tuple containing the objective value and a tuple of pressure-velocity field and 
      material fraction.
  """
  def objective_wrapper(mat_frac):
    brinkman_penalty = _mat.compute_ramp_interpolation(
      prop=mat_frac,
      ramp_penalty=ramp_penalty,
      prop_ext=inv_permeability_ext,
      mode="convex",
    )
    press_vel = _solv.modified_newton_raphson_solve(
      flow_solver, init_press_vel, brinkman_penalty
    )
    press_vel_elem = press_vel[mesh.elem_dof_mat]
    u_vel_elem = jnp.mean(press_vel_elem[:, 1::3], axis=1)

    drag = jnp.einsum("e, e -> ", u_vel_elem, brinkman_penalty.reshape((-1,)))/obj_scale
    return drag, (press_vel, mat_frac)

  (obj, (press_vel, mat_frac)), d_obj = jax.value_and_grad(
    objective_wrapper, has_aux=True
  )(mat_frac)
  return obj, d_obj.reshape((-1, 1)), press_vel, mat_frac

# Constraint

We define a simple material constraint optimization problem where we restrict the minimum material/solid the design can assume. With the user defined `min_mat_frac` we compute the material constraint. Once again, the gradients are computed automatically via jax's `value_and_grad`.

In [ ]:
def constraint_function(mat_frac: ArrayLike, min_mat_frac: float):
  """Computes the constraint for the material fraction.
  This constraint ensures that the mean material fraction is above a minimum threshold.

  Args:
    mat_frac: Material fraction array of shape (num_elems,). The array should contain 
      values between 0 and 1, representing the material fraction at each element. Where 
      0 indicates fluid and 1 indicates material. The material fraction can assume 
      intermediate values between 0 and 1 during optimization.
    min_mat_frac: Minimum material fraction threshold ensures material presence in the 
      design.

  Returns:
    A scalar value representing the constraint violation.
  """
  def constraint_wrapper(mat_frac):
    mean_mat_frac = jnp.mean(mat_frac)
    return 1.0 - (mean_mat_frac / min_mat_frac)

  mat_cons, d_mat_cons = jax.value_and_grad(constraint_wrapper)(mat_frac)
  return jnp.array([mat_cons]), d_mat_cons.T

## Optimization loop

Here, we define the optimization loop. We use the MMA optimizer. We begin the optimization with a uniform design corresponding to the minimum allowed material/solid fraction. The optimization flowchart is illustrated below:

<img src="../../figures/drag_flowchart.png" alt="flowchart" style="width: 800px;"/>

In [ ]:
def optimize_design(
  fe: _fea.FluidSolver,
  min_mat_frac: float,
  init_design: jnp.ndarray,
  max_iter: int,
  move_limit: float = 1e-2,
  kkt_tol: float = 1e-3,
  step_tol: float = 1e-3,
  plot_interval: int = 5,
):
  """Optimizes the design using the Method of Moving Asymptotes (MMA) optimization.

The function initializes the design variables with the minimum material fraction and 
  then iteratively updates them via the MMA approach. At each iteration, the objective 
  function and its gradient are computed using JAX `value_and_grad` function based 
  on the current material distribution. Additionally, a material constraint is enforced
  to ensure the design maintains at least the specified minimum material fraction. 

Args:
  fe: Fluid solver instance containing mesh, boundary conditions, and material 
    properties.
  min_mat_frac: Minimum allowed material fraction to enforce material presence in the 
    design.
  max_iter: Maximum number of iterations for the MMA optimization loop.
  move_limit: Maximum allowable change in the design variable per iteration.
  kkt_tol: Tolerance for the Karush-Kuhn-Tucker (KKT) optimality criteria.
  step_tol: Tolerance for the optimization step size.
  plot_interval: Number of iterations between plot updates.

Returns:
  mma_state: The final state of the MMA optimization, including the optimized design 
    variables, iteration count, and convergence flag.
  history: A dictionary recording the history of the objective function values and 
    constraint violations with keys:
    'obj'      - list of objective function values,
    'vol_cons' - list of volume constraint values.
"""
  init_design = np.array(init_design)
  num_design_var = init_design.shape[0]
  num_cons = 1
  lower_bound = np.zeros((num_design_var, 1))
  upper_bound = np.ones((num_design_var, 1))
  mma_params = _mma.MMAParams(
    max_iter=max_iter,
    kkt_tol=kkt_tol,
    step_tol=step_tol,
    move_limit=move_limit,
    num_design_var=num_design_var,
    num_cons=num_cons,
    lower_bound=lower_bound,
    upper_bound=upper_bound,
  )
  mma_state = _mma.init_mma(init_design.reshape((-1, 1)), mma_params)

  ramp_penalty = 10.0
  history = {"obj": [], "vol_cons": []}

  press_vel = jnp.zeros((fe.mesh.num_dofs,))
  press_vel = press_vel.at[fe.bc["fixed_dofs"]].set(fe.bc["dirichlet_values"])

  while not mma_state.is_converged:
    objective, grad_obj, press_vel, mat_frac = objective_function(
      mma_state.x, fe, ramp_penalty, press_vel
    )
    press_vel = jax.lax.stop_gradient(press_vel)
    press_vel = press_vel.at[fe.bc["fixed_dofs"]].set(fe.bc["dirichlet_values"])

    constr, grad_cons = constraint_function(mma_state.x, min_mat_frac)

    # Zero out the gradients for the wall and fluid domain
    grad_cons = grad_cons.at[0, non_design_elems].set(0.0)
    grad_obj = grad_obj.at[non_design_elems, 0].set(0.0)

    mma_state = _mma.update_mma(
      mma_state, mma_params, objective, grad_obj, constr, grad_cons
    )

    status = f"epoch {mma_state.epoch} J {objective:.2E} mc {constr[0]:.2F}"
    print(status)
    if mma_state.epoch % plot_interval == 0 or mma_state.epoch == 1:
      _, ax = plt.subplots(1, 1)
      ax = _viz.plot_grid_mesh(
        mesh=mesh,
        field=mma_state.x.reshape(-1),
        ax=ax,
        colorbar=True,
        cmap=_cmap,
        val_range=(0.0, 1.0),
      )
      ax.set_title(status)
      press_vel_elem = press_vel[mesh.elem_dof_mat]
      press_elem = np.mean(press_vel_elem[:, 0::3], axis=1)
      u_vel_elem = np.mean(press_vel_elem[:, 1::3], axis=1)
      v_vel_elem = np.mean(press_vel_elem[:, 2::3], axis=1)
      vel_elem_mag = np.sqrt(u_vel_elem**2 + v_vel_elem**2)
      _, ax = plt.subplots(1, 1)
      ax = _viz.plot_grid_mesh(mesh=mesh, field=vel_elem_mag, ax=ax, colorbar=True)
      ax.set_title("Velocity Magnitude")

      _, ax = plt.subplots(1, 1)
      ax = _viz.plot_grid_mesh(mesh=mesh, field=press_elem, ax=ax, colorbar=True)
      ax.set_title("Pressure")
      plt.show()

  return mma_state, history

In [ ]:
mma_state, u = optimize_design(
  flow_solver,
  min_mat_frac=min_mat_frac,
  init_design=init_design,
  max_iter=250,
  move_limit=5.0e-2,
)